# ML Spaceship Titanic
After solving titanic, we try solving https://www.kaggle.com/c/spaceship-titanic/data

### Imports and converting file to dataframe

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier

In [8]:

# Loading data into pandas dataframe
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

count = 0
for i in train["Transported"]:
    if i==1:
        count +=1 

# Figuring out some key characteristics of the data
print("Survival rate:", count/len(train["Transported"]))

le = len(train)
train_dropna = train.dropna()
print("data length", le, "-->", len(train_dropna), "if we drop NA")
print()
print(train.head())
print(train.shape)
print(test.shape)

Survival rate: 0.5036236051995858
data length 8693 --> 6606 if we drop NA

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        Fal

### Preparing dataset

I filled in the NaN with the median/mean/mode, and converted categorical data to T/F data by using one-hot-encoding. I used the standard scalar (built into scikit learn) to scale the numercal data.


In [24]:
def process_data(df):
    #replace missing values
    replacements = {"HomePlanet":df['HomePlanet'].mode(),
                    "CryoSleep":df['CryoSleep'].mode(),
                    "Cabin":"U/0/U", #U for unknown
                    "Destination": df['CryoSleep'].mode(),
                    "Age": df['Age'].median(),
                    "VIP": df['VIP'].mode(),
                    "RoomService": df['RoomService'].mode(),
                    "FoodCourt": df['FoodCourt'].mode(),
                    "ShoppingMall": df['ShoppingMall'].mode(),
                    "Spa": df['Spa'].mode(),
                    "VRDeck": df['VRDeck'].mode(),
                    #If last name is the same they might be related... not sure how to handle that
                    }
    
    df = df.fillna(value=replacements)  #this doesn't modify inplace!

    df.dropna(subset=["Cabin"])
    # Convert cabin to 3 numbers
    
    df['deck'] = df['Cabin'].map(lambda x: str(x).split("/")[0])
    df['num'] = df['Cabin'].map(lambda x: str(x).split("/")[1])
    df['side'] = df['Cabin'].map(lambda x: str(x).split("/")[2])

    #Drop columns
    df = df.drop(columns=["PassengerId", "Name", "Cabin"]) 

    # one-hot-encoding
    df = pd.get_dummies(df, columns=['HomePlanet', 'Destination', "deck", "side"])

    #last resort...
    df = df.dropna()
    return df

train1 = process_data(train)
test1 = process_data(test)
labels = train1["Transported"]
train1 = train1.drop(columns = ["Transported"]) #This is y

print(train1.columns)
print(test1.columns)
# print(labels.head())
print(train1.shape)
print(test1.shape)
print(labels.shape)

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'num', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_T',
       'deck_U', 'side_P', 'side_S', 'side_U'],
      dtype='object')
Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'num', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_T',
       'deck_U', 'side_P', 'side_S', 'side_U'],
      dtype='object')
(7409, 27)
(3652, 27)
(7409,)


**Scaling**
scaling numerical data using standard scalar

In [25]:

from sklearn.preprocessing import StandardScaler
scalar_age = StandardScaler() 
scalar_fare = StandardScaler() #need two seperate scalars!

train2 = train1
test2 = test1

# List of columns to scale
columns = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

# Scale each column individually and store the scaler
for col in columns:
    scaler = StandardScaler()
    train2[col] = scaler.fit_transform(train1[[col]])

for col in columns:
    scaler = StandardScaler()
    test2[col] = scaler.fit_transform(test1[[col]])

In [26]:
y = labels
X = train2
Z = test2
print(X.columns)
print(y.head(5))
print(Z.columns)
print(X.shape)
print(Z.shape)

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'num', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_T',
       'deck_U', 'side_P', 'side_S', 'side_U'],
      dtype='object')
0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool
Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'num', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'deck_A',
       'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'deck_T',
       'deck_U', 'side_P', 'side_S', 'side_U'],
      dtype='object')
(7409, 27)
(3652, 27)


## Fitting to model

### KNN
The k-nearest neighbour method was used; GridSearchCV was used to search for the best k-parameter

In [42]:
knn = KNeighborsRegressor()
knn_mod = GridSearchCV(estimator = knn,
             param_grid = {'n_neighbors': list(range(14, 37))})
knn_mod.fit(X, y)
results = pd.DataFrame(knn_mod.cv_results_)

In [ ]:
print(pd.DataFrame(results[["param_n_neighbors","rank_test_score"]]))
# best results when k=25
pred_knn = knn_mod.predict(X)
pred_knn = [int(x) for x in pred_knn]
from sklearn.metrics import accuracy_score
print("The percentage accuracy is", f'{100* accuracy_score(pred_knn, y):.2f}')

    param_n_neighbors  rank_test_score
0                  14               23
1                  15               22
2                  16               21
3                  17               19
4                  18               20
5                  19               18
6                  20               13
7                  21               14
8                  22               10
9                  23               17
10                 24               15
11                 25                1
12                 26               11
13                 27               16
14                 28                6
15                 29                2
16                 30                3
17                 31                9
18                 32                5
19                 33                4
20                 34                8
21                 35                7
22                 36               12
The percentage accuracy is 49.78


### Neural Network

I tested nerual networks with 1 and 2 hidden layers

In [38]:
from sklearn.neural_network import MLPClassifier

In [45]:
clf = MLPClassifier(solver='lbfgs', alpha=1, random_state=1, max_iter=3000)
NN_mod_one_layer = GridSearchCV(estimator = clf,
             param_grid = {'hidden_layer_sizes': (range(10, 20))})  
NN_mod_one_layer.fit(X, y)


GridSearchCV(estimator=MLPClassifier(alpha=1, max_iter=3000, random_state=1,
                                     solver='lbfgs'),
             param_grid={'hidden_layer_sizes': range(10, 20)})

In [47]:
def iterate_layers(min_i, max_i, min_j, max_j):
    for i in range(min_i, max_i):
        for j in range(min_j, max_j): 
            yield (i,j)

NN_mod_two_layer = GridSearchCV(estimator = clf,
             param_grid = {'hidden_layer_sizes': list(iterate_layers(20,25,5,10))})
NN_mod_two_layer.fit(X, y)

/Users/for_phone/opt/anaconda3/envs/sklearn-env/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/for_phone/opt/anaconda3/envs/sklearn-env/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


GridSearchCV(estimator=MLPClassifier(alpha=1, max_iter=3000, random_state=1,
                                     solver='lbfgs'),
             param_grid={'hidden_layer_sizes': [(20, 5), (20, 6), (20, 7),
                                                (20, 8), (20, 9), (21, 5),
                                                (21, 6), (21, 7), (21, 8),
                                                (21, 9), (22, 5), (22, 6),
                                                (22, 7), (22, 8), (22, 9),
                                                (23, 5), (23, 6), (23, 7),
                                                (23, 8), (23, 9), (24, 5),
                                                (24, 6), (24, 7), (24, 8),
                                                (24, 9)]})

In [48]:
print(pd.DataFrame(NN_mod_one_layer.cv_results_)[["param_hidden_layer_sizes", "rank_test_score"]])
print(pd.DataFrame(NN_mod_two_layer.cv_results_)[["param_hidden_layer_sizes", "rank_test_score"]])

   param_hidden_layer_sizes  rank_test_score
0                        10                8
1                        11                9
2                        12               10
3                        13                1
4                        14                4
5                        15                3
6                        16                2
7                        17                5
8                        18                7
9                        19                6
   param_hidden_layer_sizes  rank_test_score
0                   (20, 5)               25
1                   (20, 6)               13
2                   (20, 7)               20
3                   (20, 8)                5
4                   (20, 9)                4
5                   (21, 5)               14
6                   (21, 6)               24
7                   (21, 7)               21
8                   (21, 8)                8
9                   (21, 9)                2
10        

In [49]:
from sklearn.metrics import accuracy_score

pred_NN_mod = NN_mod_one_layer.predict(X)
print("The best percentage accuracy is", f'{100* accuracy_score(pred_NN_mod, y):.2f}', "one layer, with hidden_layer = 13")

pred_NN_mod = NN_mod_two_layer.predict(X)
print("The best percentage accuracy is", f'{100* accuracy_score(pred_NN_mod, y):.2f}', "two layers, with hidden_layer = (22, 9)")

The best percentage accuracy is 79.63 one layer, with hidden_layer = 13
The best percentage accuracy is 80.29 two layers, with hidden_layer = (22, 9)


## Decision Tree

In [ ]:
mod_dtree = DecisionTreeClassifier().fit(X, y)
pred_dtree_mod = mod_dtree.predict(X)
print("The accuracy score is", f'{100* accuracy_score(pred_dtree_mod, y):.2f}', ", high as expected")

The accuracy score is 98.32 , high as expected


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_forest = RandomForestClassifier(n_estimators=10)
mod_forest = GridSearchCV(estimator = clf_forest, param_grid={"n_estimators": list(range(3, 15))})
mod_forest.fit(X, y)
pred_forest = mod_forest.predict(X)


In [ ]:
print(pd.DataFrame(mod_forest.cv_results_)[["param_n_estimators", "rank_test_score"]])
print("The best percentage accuracy is", f'{100* accuracy_score(pred_forest, y):.2f}')

    param_n_estimators  rank_test_score
0                    3               12
1                    4                6
2                    5               11
3                    6                4
4                    7               10
5                    8                8
6                    9                9
7                   10                2
8                   11                1
9                   12                5
10                  13                7
11                  14                3
The best percentage accuracy is 97.87


In [ ]:
#extra random

from sklearn.ensemble import ExtraTreesClassifier

clf_extraTree = ExtraTreesClassifier()
mod_extraTree = GridSearchCV(estimator = clf_extraTree, param_grid={"n_estimators": list(range(3, 10))})
mod_extraTree.fit(X, y)
pred_extraTree = mod_extraTree.predict(X)

In [ ]:
print(pd.DataFrame(mod_extraTree.cv_results_)[["param_n_estimators", "rank_test_score"]])
print("The best percentage accuracy is", f'{100* accuracy_score(pred_extraTree, y):.2f}')

   param_n_estimators  rank_test_score
0                   3                7
1                   4                1
2                   5                3
3                   6                2
4                   7                4
5                   8                5
6                   9                6
The best percentage accuracy is 98.32


## Final Prediction
1. The training accuracy was high for Neural network with (8,3), which I submitted this to kaggle. 
The testing accuracy was 0.76315 though...
2. The best KNN data
3. decision tree

In [ ]:
pred_final = NN_mod_two_layer.predict(Z)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived":pred_final

})
print(df)
df.to_csv('resultNN2layer.csv', index=False)  

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [ ]:
pred_final_one_layer = NN_mod_one_layer.predict(Z)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived":pred_final_one_layer
})
df.to_csv('resultNN1layer.csv', index=False)  

In [ ]:
pred_final_knn = knn_mod.predict(Z)
pred_final_knn = np.round(pred_final_knn).astype(int)  # This ensures this is an integer array (says ChatGPT)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived": pred_final_knn
})
df.to_csv('resultKNN.csv', index=False) 

In [ ]:
pred_final_dtree = mod_dtree.predict(Z)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived":pred_final_dtree
})
df.to_csv('resultDTree.csv', index=False)  

In [ ]:
pred_final_forest = mod_forest.predict(Z)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived":pred_final_forest
})
df.to_csv('resultForest.csv', index=False)  

In [ ]:
pred_final_extraTree = mod_extraTree.predict(Z)
df = pd.DataFrame({ "PassengerId":range(892, 1310),
                   "Survived":pred_final_extraTree
})
df.to_csv('resultExtraTree.csv', index=False)  